In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, cross_val_predict
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier
from scipy import stats
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import lightgbm as LGBMClassifier
from sklearn import *
from lightgbm import LGBMClassifier
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

import plotly.express as px
import plotly.graph_objects as go

sns.set()

In [3]:
tdata=pd.read_csv("train.csv")
df=tdata.copy()
test_df=pd.read_csv("test.csv")
dataset=pd.concat([df,test_df])

In [4]:
dataset

,ID,sequence1,sequence2,sequence3,sequence4,sequence5,sequence6,sequence7,sequence8,sequence9,...,M,N,O,P,Q,R,S,T,U,mutation
0,0,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0.0
1,2,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0.0
2,5,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,...,0.524760,0.580085,0.301498,0.261055,0.193988,0.935688,0.685868,0.277694,0.428115,0.0
3,7,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATTCCATATTTT,AAATCAGCTAAA,TAACATACAGAG,TAACATACAGAG,...,0.943545,0.209744,0.670577,0.772650,0.845751,0.507320,0.927204,0.605755,0.454155,1.0
4,9,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,GATCGTAGCTAA,GATCGTAGCTAA,ATTCCATATTTT,AAATCAGCTAAA,ATTCCATATTTT,TAACATACAGAG,...,0.397622,0.506096,0.200673,0.734145,0.521673,0.365571,0.416094,0.584799,0.447149,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,299983,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,ATTCCATATTTT,...,0.260891,0.390874,0.208736,0.799856,0.944627,0.235131,0.303993,0.726861,0.385247,NaN
119996,299984,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATTCCATATTTT,AAATCAGCTAAA,AAATCAGCTAAA,AAATCAGCTAAA,...,0.461298,0.228659,0.266070,0.079081,0.210897,0.453764,0.416162,0.384102,0.337177,NaN
119997,299990,ATGCGCTTAATT,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,GATCGTAGCTAA,ATTCCATATTTT,AAATCAGCTAAA,TAACATACAGAG,AAATCAGCTAAA,...,0.238612,0.269438,0.592967,0.147710,0.721238,0.264155,0.683730,0.265648,0.409764,NaN
119998,299991,ATGCGCTTAATT,ATGCGCTTAATT,GATCGTAGCTAA,ATGCGCTTAATT,ATGCGCTTAATT,AAATCAGCTAAA,ATTCCATATTTT,ATTCCATATTTT,AAATCAGCTAAA,...,0.309847,0.597138,0.365207,0.489361,0.524614,0.250419,0.419844,0.454539,0.312508,NaN


In [5]:
ds=dataset.copy()
for i in ds.columns:
    if ds[i].dtype=="O":
        ds[i] = pd.Categorical(ds[i]).codes

In [6]:
X=ds.drop(columns=["mutation"])
#X=ds[sp]
y=ds['mutation']

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [17]:
X_train = X[:170000]
y_train = y[:170000]

In [18]:
y_train=y_train.astype(int)

In [19]:
X_test=X[180000:]
X_test.shape

(120000, 31)

In [11]:
lr = CatBoostClassifier()
lr.fit(X_train,y_train)

Learning rate set to 0.087524
0:	learn: 0.6363433	total: 207ms	remaining: 3m 26s
1:	learn: 0.5904848	total: 252ms	remaining: 2m 5s
2:	learn: 0.5517140	total: 282ms	remaining: 1m 33s
3:	learn: 0.5232717	total: 310ms	remaining: 1m 17s
4:	learn: 0.4994011	total: 340ms	remaining: 1m 7s
5:	learn: 0.4779625	total: 372ms	remaining: 1m 1s
6:	learn: 0.4613895	total: 404ms	remaining: 57.3s
7:	learn: 0.4482808	total: 440ms	remaining: 54.5s
8:	learn: 0.4375149	total: 472ms	remaining: 52s
9:	learn: 0.4283091	total: 500ms	remaining: 49.5s
10:	learn: 0.4199764	total: 531ms	remaining: 47.8s
11:	learn: 0.4134506	total: 562ms	remaining: 46.3s
12:	learn: 0.4086094	total: 593ms	remaining: 45s
13:	learn: 0.4044576	total: 628ms	remaining: 44.2s
14:	learn: 0.4007388	total: 660ms	remaining: 43.4s
15:	learn: 0.3971080	total: 692ms	remaining: 42.5s
16:	learn: 0.3940267	total: 724ms	remaining: 41.8s
17:	learn: 0.3916647	total: 756ms	remaining: 41.3s
18:	learn: 0.3893776	total: 791ms	remaining: 40.8s
19:	learn: 0

161:	learn: 0.3526050	total: 5.64s	remaining: 29.2s
162:	learn: 0.3524959	total: 5.68s	remaining: 29.2s
163:	learn: 0.3524109	total: 5.72s	remaining: 29.1s
164:	learn: 0.3523027	total: 5.75s	remaining: 29.1s
165:	learn: 0.3521930	total: 5.8s	remaining: 29.1s
166:	learn: 0.3520981	total: 5.84s	remaining: 29.1s
167:	learn: 0.3520222	total: 5.87s	remaining: 29.1s
168:	learn: 0.3518993	total: 5.92s	remaining: 29.1s
169:	learn: 0.3518315	total: 5.95s	remaining: 29.1s
170:	learn: 0.3517268	total: 6s	remaining: 29.1s
171:	learn: 0.3516186	total: 6.04s	remaining: 29.1s
172:	learn: 0.3515241	total: 6.07s	remaining: 29s
173:	learn: 0.3514240	total: 6.1s	remaining: 28.9s
174:	learn: 0.3513355	total: 6.13s	remaining: 28.9s
175:	learn: 0.3512462	total: 6.16s	remaining: 28.8s
176:	learn: 0.3511598	total: 6.2s	remaining: 28.8s
177:	learn: 0.3510761	total: 6.23s	remaining: 28.8s
178:	learn: 0.3510000	total: 6.26s	remaining: 28.7s
179:	learn: 0.3508780	total: 6.29s	remaining: 28.7s
180:	learn: 0.350798

321:	learn: 0.3403782	total: 10.9s	remaining: 23s
322:	learn: 0.3403206	total: 10.9s	remaining: 22.9s
323:	learn: 0.3402830	total: 11s	remaining: 22.9s
324:	learn: 0.3402356	total: 11s	remaining: 22.9s
325:	learn: 0.3401769	total: 11s	remaining: 22.8s
326:	learn: 0.3401091	total: 11.1s	remaining: 22.8s
327:	learn: 0.3400679	total: 11.1s	remaining: 22.7s
328:	learn: 0.3400086	total: 11.1s	remaining: 22.7s
329:	learn: 0.3399394	total: 11.2s	remaining: 22.6s
330:	learn: 0.3399004	total: 11.2s	remaining: 22.6s
331:	learn: 0.3398371	total: 11.2s	remaining: 22.6s
332:	learn: 0.3397776	total: 11.2s	remaining: 22.5s
333:	learn: 0.3397206	total: 11.3s	remaining: 22.5s
334:	learn: 0.3396860	total: 11.3s	remaining: 22.4s
335:	learn: 0.3396129	total: 11.3s	remaining: 22.4s
336:	learn: 0.3395397	total: 11.4s	remaining: 22.4s
337:	learn: 0.3394651	total: 11.4s	remaining: 22.3s
338:	learn: 0.3393927	total: 11.4s	remaining: 22.3s
339:	learn: 0.3393279	total: 11.5s	remaining: 22.3s
340:	learn: 0.339262

481:	learn: 0.3311105	total: 16s	remaining: 17.2s
482:	learn: 0.3310798	total: 16s	remaining: 17.1s
483:	learn: 0.3310097	total: 16.1s	remaining: 17.1s
484:	learn: 0.3309624	total: 16.1s	remaining: 17.1s
485:	learn: 0.3309142	total: 16.1s	remaining: 17s
486:	learn: 0.3308612	total: 16.1s	remaining: 17s
487:	learn: 0.3308128	total: 16.2s	remaining: 17s
488:	learn: 0.3307422	total: 16.2s	remaining: 16.9s
489:	learn: 0.3306843	total: 16.2s	remaining: 16.9s
490:	learn: 0.3306561	total: 16.3s	remaining: 16.9s
491:	learn: 0.3305861	total: 16.3s	remaining: 16.8s
492:	learn: 0.3305354	total: 16.3s	remaining: 16.8s
493:	learn: 0.3304755	total: 16.4s	remaining: 16.8s
494:	learn: 0.3304137	total: 16.4s	remaining: 16.7s
495:	learn: 0.3303584	total: 16.4s	remaining: 16.7s
496:	learn: 0.3303066	total: 16.5s	remaining: 16.7s
497:	learn: 0.3302625	total: 16.5s	remaining: 16.6s
498:	learn: 0.3302180	total: 16.5s	remaining: 16.6s
499:	learn: 0.3301686	total: 16.5s	remaining: 16.5s
500:	learn: 0.3301136	

KeyboardInterrupt: 

In [ ]:
pred=lr.predict(X_test)

In [38]:
importance=lr.feature_importances_


In [59]:
importance=pd.DataFrame(importance)
features=pd.DataFrame(ds.columns)

feat=pd.concat([features,importance],axis=1)
feat.columns=["columns","importance"]
cols=feat[feat.importance>1]["columns"].tolist()

In [60]:
len(cols)

28

In [ ]:
from sklearn.naive_bayes import GaussianNB
GaussianNB()

In [12]:
X=ds.drop(columns=["mutation"])

y=ds['mutation']

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [20]:

X_train = X[:170000]
y_train = y[:170000]
y_train=y_train.astype(int)
X_test=X[180000:]

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
no_splits = 10
kfold = KFold(n_splits=no_splits, shuffle=True, random_state=1)
counter = 0
models = {}
scores = []
model_s=[]
for train, test in kfold.split(X_train):
    counter += 1
    print(train.shape, test.shape)
    train_X, test_X = X_train[train], X_train[test]
    train_y, test_y = y_train[train], y_train[test]
    print(train_y.shape, train_y.shape)
    print("end")
#   fit model
    model = DummyClassifier()
    #print(train_X,train_y)
    model.fit(train_X, train_y)
    models[f"model{str(counter)}"] = model
#   evaluate model
    yhat = model.predict(test_X)
    acc = accuracy_score(test_y, yhat)
    model_s.append(model)
    scores.append(acc)

(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end
(153000,) (17000,)
(153000,) (153000,)
end


In [24]:



scores=pd.DataFrame(scores)
scores

,0
0,0.734647
1,0.735294
2,0.738294
3,0.737294
4,0.739765
5,0.735647
6,0.727941
7,0.737353
8,0.731294
9,0.732765


In [30]:
models

{'model1': LGBMClassifier(),
 'model2': LGBMClassifier(),
 'model3': LGBMClassifier(),
 'model4': LGBMClassifier(),
 'model5': LGBMClassifier(),
 'model6': LGBMClassifier(),
 'model7': LGBMClassifier(),
 'model8': LGBMClassifier(),
 'model9': LGBMClassifier(),
 'model10': LGBMClassifier()}

In [31]:

y_test=y[180000:]
pred=models["model5"].predict(X_test)
pred=pd.DataFrame(pred)
test=pd.read_csv("test.csv")
y_pred=pd.concat([test["ID"],pred],axis=1)
y_pred.columns=["ID","mutation"]
y_pred


,ID,mutation
0,1,0
1,3,0
2,4,0
3,6,0
4,8,0
...,...,...
119995,299983,0
119996,299984,1
119997,299990,1
119998,299991,0


In [32]:
y_pred.to_csv('lgbm.csv', index = False)